In [2]:
import shutil, os, warnings, re, datetime, multiprocessing
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import tensorflow as tf
import _pickle as pickle
from gensim.models import Doc2Vec
from itertools import chain, combinations
from sklearn.model_selection import train_test_split
from collections import namedtuple, Counter
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings(action='default')

In [3]:
# start = datetime.datetime.now()
# 주문_2014 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2014.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문_2015 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2015.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문_2016 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2016.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문_2017 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2017.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문_2018 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2018.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품_2014 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2014.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품_2015 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2015.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품_2016 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2016.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품_2017 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2017.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품_2018 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2018.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품출하_2014 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2014.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품출하_2015 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2015.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품출하_2016 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2016.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품출하_2017 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2017.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 주문상품출하_2018 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2018.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
# 상품마스터 = pd.read_csv('c:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/상품마스터.csv',encoding ='utf-8',dtype = 'str')
# 조직마스터 = pd.read_csv('c:/users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/조직마스터.csv', encoding ='utf-8', dtype ='str')
# 사업장= pd.read_csv('c:/users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/사업장.csv', encoding ='utf-8', dtype ='str')
# 공사유형 = pd.read_csv('c:/users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/공사유형.csv', encoding ='utf-8', dtype ='str')

# 주문 = pd.concat([주문_2014, 주문_2015, 주문_2016, 주문_2017, 주문_2018])
# 주문상품 = pd.concat([주문상품_2014, 주문상품_2015, 주문상품_2016, 주문상품_2017, 주문상품_2018])
# 주문상품출하 = pd.concat([주문상품출하_2014, 주문상품출하_2015, 주문상품출하_2016, 주문상품출하_2017, 주문상품출하_2018])

# # 주문 처리
# 주문 = 주문.reset_index()[['ORDE_IDEN_NUMB','CONS_IDEN_NAME','GROUPID','CLIENTID','BRANCHID',
#                        'DELI_AREA_CODE','REGI_DATE_TIME','ORDE_USER_ID']].drop_duplicates(keep='first')
# 주문상품 = 주문상품.reset_index()[['ORDE_IDEN_NUMB','ORDE_SEQU_NUMB','GOOD_IDEN_NUMB',
#                            'ORDE_REQU_QUAN']].drop_duplicates(keep='first')
# 주문상품 = 주문상품.groupby(['ORDE_IDEN_NUMB','GOOD_IDEN_NUMB'])['ORDE_REQU_QUAN'].agg('sum').reset_index()
# 주문상품출하 =  주문상품출하.reset_index()[['ORDE_IDEN_NUMB', 'DELI_STAT_FLAG']].drop_duplicates(keep='first')
# 주문상품출하 = 주문상품출하[주문상품출하.DELI_STAT_FLAG=='70'].ORDE_IDEN_NUMB.unique()

# 주문상품_주문 = pd.merge(주문상품, 주문, on = 'ORDE_IDEN_NUMB', how = 'left')
# 주문상품_주문_출하 = 주문상품_주문[주문상품_주문.ORDE_IDEN_NUMB.isin(주문상품출하)]
# 주문전체 = 주문상품_주문_출하[주문상품_주문_출하.GROUPID != '101'].drop_duplicates(keep='first')

# # 상품 처리
# 상품마스터 = 상품마스터[['good_iden_numb','cate_id','good_name','good_spec','good_type','repre_good']].drop_duplicates(keep='first')
# 상품마스터 = 상품마스터.rename(columns = {'good_iden_numb':'GOOD_IDEN_NUMB'})
# # repre_good - Y : 옵션대표상품, N : 단품, P : 옵션상품 // good_type - 10 : 일반, 20 : 지정,60 : 공구, 70 : 안전,80 : 보안

# 상품주문전체 = pd.merge(주문전체, 상품마스터, on = 'GOOD_IDEN_NUMB', how = 'left')
# 상품주문전체 = 상품주문전체[상품주문전체.repre_good=='N'] # 옵션상품 제외

# # 사업장 처리
# 사업장 = pd.concat([
#         사업장[['BRANCHID','AREATYPE','BRANCHBUSITYPE','BRANCHBUSICLAS','WORKID']].rename(columns={'BRANCHID':'BORGID'}), 
#         사업장[['BRANCHCD','AREATYPE','BRANCHBUSITYPE','BRANCHBUSICLAS','WORKID']].rename(columns={'BRANCHCD':'BORGID'})
#     ], axis=0).drop_duplicates(keep='first')

# 공사유형_사업장 = pd.merge(
#     사업장, 
#     공사유형[['WORKID','WORKNM']].drop_duplicates(keep='first'), 
#     how = 'left', on = 'WORKID')

# 조직마스터 = 조직마스터[(조직마스터.BORGTYPECD == 'BCH') & (조직마스터.SVCTYPECD == 'BUY')] # 사업장레벨 및 구매사만
# 조직마스터 = pd.concat([
#         조직마스터[['BORGID','BORGNM']], 
#         조직마스터[['BORGCD','BORGNM']].rename(columns={'BORGCD':'BORGID'})
#     ], axis=0).drop_duplicates(keep='first')

# 조직전체 = pd.merge(조직마스터, 공사유형_사업장, how = 'left', on = 'BORGID').rename(columns={'BORGID':'BRANCHID'})

# df = pd.merge(상품주문전체,조직전체,how = 'left',on = 'BRANCHID')
# # 불필요컬럼 제외
# df = df.drop([
#         'CONS_IDEN_NAME','GROUPID','CLIENTID','ORDE_USER_ID','good_name','good_spec','repre_good','WORKNM','BORGNM'
#         ], axis = 1)
# df = df.drop_duplicates(keep='first')
# # 날짜컬럼추가
# df["REGI_DATE"] = pd.to_datetime(df.REGI_DATE_TIME).dt.date
# # 문자 -> 숫자화1
# bptype = LabelEncoder()
# df['BpType']=bptype.fit_transform(df.BRANCHBUSITYPE.tolist()).astype('str')
# with open('./le_BpType.pkl','wb') as f:
#     pickle.dump(bptype, f)
# # 문자 -> 숫자화2
# bpclass = LabelEncoder()
# df['BpClass']=bpclass.fit_transform(df.BRANCHBUSICLAS.tolist()).astype('str')
# with open('./le_BpClass.pkl','wb') as g:
#     pickle.dump(bpclass, g)
# # 널 데이터 제외(764건)
# df = df.dropna()
# df = df.drop(['BRANCHBUSITYPE','BRANCHBUSICLAS'],axis =1)
# df.columns = ['OrderNum','ProductCode','ProductAmt','BpID', 'Deli_Region','OrderTime','ProductCategory','ProductClass','Region',
#               'ConstructionType','OrderDate','BpType','BpClass']
# duration = datetime.datetime.now()-start
# m, s = divmod(duration.seconds, 60);h, m = divmod(m, 60);print("[%02d:%02d:%02d]" %(h, m, s))
# df = df[['OrderNum','OrderTime','OrderDate','ProductCode','ProductCategory', 'ProductClass','ProductAmt', 
#     'Deli_Region','BpID','Region','ConstructionType','BpType', 'BpClass']]
# # 이상 데이터 불러오기 및 필요 데이터 필터링 부분.
# df.to_csv('./Datasets/dfdf_181109_new.csv', index = False)

In [3]:
df = pd.read_csv('C:/pythondata/widendepp/df_181109_new.csv', dtype='str', encoding = 'utf-8')

In [4]:
df.head()

,OrderNum,OrderTime,OrderDate,ProductCode,ProductCategory,ProductClass,ProductAmt,Deli_Region,BpID,Region,ConstructionType,BpType,BpClass
0,001404030001,2014-04-03 17:15:08.153,2014-04-03,10000001489,100761,20,1200,14,304956,10,9,53,81
1,001405270001,2014-05-27 09:29:59.153,2014-05-27,7400800012,100761,20,500,14,304956,10,9,53,81
2,001406030002,2014-06-03 17:30:24.200,2014-06-03,7400800012,100761,20,1500,14,304956,10,9,53,81
3,001410160001,2014-10-16 09:58:55.273,2014-10-16,10000001488,100761,20,600,14,304956,10,9,53,81
4,001411050001,2014-11-05 11:40:57.543,2014-11-05,10000001489,100761,20,1200,14,304956,10,9,53,81


In [5]:
df['OrderTime'] =  pd.to_datetime(df['OrderTime'])
df['OrderDate'] =  pd.to_datetime(df['OrderDate'])
df['OrderYear'] = df['OrderDate'].dt.year
df['ProductAmt'] =  df['ProductAmt'].astype('int64')
df = df.drop(['OrderNum', 'OrderTime', 'Deli_Region','ProductAmt','ProductCategory','ProductClass'],axis = 1)

df1 = df.groupby(['OrderDate','BpID','Region','ConstructionType','BpType','BpClass','OrderYear'])['ProductCode'].apply(list).reset_index()

In [7]:
df1.head()

,OrderDate,BpID,Region,ConstructionType,BpType,BpClass,OrderYear,ProductCode
0,2014-01-02,304660,41,5,7,205,2014,"[10000000332, 10000000334, 10000000338, 100000..."
1,2014-01-02,304685,20,20,190,3,2014,"[10000001380, 10000001455, 10000001507]"
2,2014-01-02,304695,20,36,213,149,2014,"[10000002450, 10000002497, 10000002501, 100000..."
3,2014-01-02,304809,30,16,165,96,2014,"[10000008077, 10000008077]"
4,2014-01-02,304823,10,16,165,96,2014,"[10000008077, 10000008077]"


In [11]:
df2 = df1.sort_values(['BpID','OrderDate']).reset_index(drop = True)

(104326, 8)


In [13]:
df2.head()

,OrderDate,BpID,Region,ConstructionType,BpType,BpClass,OrderYear,ProductCode
0,2017-03-13,304573,11,3,149,21,2017,"[10000023533, 10000056705]"
1,2014-03-05,304576,12,7,161,96,2014,"[6601000010, 6601000024, 6601000053, 660100006..."
2,2014-04-02,304576,12,7,161,96,2014,"[10000008077, 6601000039, 6601000041, 66010000..."
3,2014-07-04,304576,12,7,161,96,2014,[6601000001]
4,2014-08-01,304576,12,7,161,96,2014,"[10000009978, 10000010251, 6601000041, 6601000..."


In [12]:
print(df2.shape)

(104326, 8)


In [14]:
df2.iloc[0:4,].reset_index(drop=True)

,OrderDate,BpID,Region,ConstructionType,BpType,BpClass,OrderYear,ProductCode
0,2017-03-13,304573,11,3,149,21,2017,"[10000023533, 10000056705]"
1,2014-03-05,304576,12,7,161,96,2014,"[6601000010, 6601000024, 6601000053, 660100006..."
2,2014-04-02,304576,12,7,161,96,2014,"[10000008077, 6601000039, 6601000041, 66010000..."
3,2014-07-04,304576,12,7,161,96,2014,[6601000001]


In [15]:
tmp = df2.iloc[0:4,].reset_index(drop=True)

In [18]:
tmp_pc = tmp['ProductCode'].tolist()

In [20]:
tmp_pc

[['10000023533', '10000056705'],
 ['6601000010',
  '6601000024',
  '6601000053',
  '6601000067',
  '6601000170',
  '6601000171',
  '6601000174',
  '6601000196'],
 ['10000008077',
  '6601000039',
  '6601000041',
  '6601000045',
  '6601000049',
  '6601000050',
  '6601000055',
  '6601000060',
  '6601000067',
  '6601000174',
  '6601000196'],
 ['6601000001']]

In [17]:
tmp['BpID'][0]

'304573'

In [30]:
tmp_pc[3:4]

[['6601000001']]

In [19]:
list(set(chain.from_iterable(tmp_pc[0:3])))

['6601000050',
 '6601000055',
 '6601000174',
 '10000008077',
 '6601000053',
 '6601000045',
 '6601000049',
 '6601000170',
 '6601000067',
 '6601000010',
 '6601000039',
 '6601000171',
 '6601000024',
 '6601000196',
 '10000023533',
 '6601000060',
 '10000056705',
 '6601000041']

In [21]:
list(set(chain.from_iterable(tmp_pc[3:4])))

['6601000001']

In [22]:
nx =  3 # 이전 기록 갯수(X)
ny = 1 # 이후 기록 갯수(Y)
df2_1 = { 'BpID':[],  'x_ProductCodes':[],  'y_ProductCodes':[]}
print('Loop start!!')
start = datetime.datetime.now()
for ii in range(len(df2)-nx):
    targetdata = df2.iloc[ii:ii+nx+ny,].reset_index(drop=True)
    if len(targetdata['BpID'].unique())==1 and len(targetdata) == nx+ny:
        tmpdata1 = targetdata['ProductCode'].tolist()
        df2_1['BpID'].append(targetdata['BpID'][0])
        df2_1['x_ProductCodes'].append(list(set(chain.from_iterable(tmpdata1[0:nx]))))
        df2_1['y_ProductCodes'].append(list(set(chain.from_iterable(tmpdata1[nx:nx+ny]))))
    if ii % 10000 == 0:
        duration = datetime.datetime.now()-start;m, s = divmod(duration.seconds, 60)
        h, m = divmod(m, 60);print("[%d]/[%d]-----[%02d:%02d:%02d]" %(ii, len(df2)-nx, h, m, s)) 

Loop start!!
[0]/[104323]-----[00:00:00]
[10000]/[104323]-----[00:00:04]
[20000]/[104323]-----[00:00:08]
[30000]/[104323]-----[00:00:13]
[40000]/[104323]-----[00:00:17]
[50000]/[104323]-----[00:00:22]
[60000]/[104323]-----[00:00:26]
[70000]/[104323]-----[00:00:31]
[80000]/[104323]-----[00:00:35]
[90000]/[104323]-----[00:00:40]
[100000]/[104323]-----[00:00:44]


In [23]:
df2_2 = pd.DataFrame(df2_1)

In [24]:
df2_2

,BpID,x_ProductCodes,y_ProductCodes
0,304576,"[6601000050, 6601000055, 6601000174, 100000080...","[6602300021, 6601000174, 6601000045, 100000102..."
1,304576,"[6601000050, 6601000055, 6601000174, 660230002...","[10000009271, 6600200174, 7400900162]"
2,304576,"[6602300021, 6601000174, 6601000045, 100000102...","[6601000174, 6602300001, 6601000067, 100000076..."
3,304576,"[6602300021, 6601000174, 6601000045, 100000102...",[10000010251]
4,304576,"[6601000174, 10000010251, 10000009271, 6602300...","[6601000060, 10000009271, 10000007607]"
5,304576,"[6601000174, 10000010251, 10000009271, 6602300...",[6601000001]
6,304576,"[10000010251, 10000009271, 10000007607, 660100...","[6601000174, 10000009978, 6601000067, 66010000..."
7,304576,"[6601000174, 10000009978, 10000009271, 6601000...","[6601000174, 6601000043, 10000010251, 66010000..."
8,304576,"[6601000174, 6601000043, 10000010251, 66010000...","[10000010251, 6601000058, 6602300012, 66010000..."
9,304576,"[6601000174, 6601000043, 10000010251, 66010000...","[6601000001, 6601000045, 10000010251]"


In [26]:
df2_2.shape

(101164, 3)

In [14]:
# # # Doc2Vec 임베딩 생성
# x_d2v_embed = myDoc2Vec(data=df2_2[['BpID','x_ProductCodes']], d2v_size=300, trainTF=False,
#                         model_loc='./Trained_models/Models_Doc2Vec/BpID_300_181022.model')
# print('x_d2v:', x_d2v_embed.shape)

x_d2v: (101164, 300)


In [31]:
# x아이템은 int로 변환
df3_1 = pd.DataFrame([list([df2_2.BpID[ii], list(map(int, df2_2.x_ProductCodes[ii]))]) for ii in df2_2.x_ProductCodes.index], 
                      columns=['BpID','x_ProductCodes'])

In [32]:
df3_1

,BpID,x_ProductCodes
0,304576,"[6601000050, 6601000055, 6601000174, 100000080..."
1,304576,"[6601000050, 6601000055, 6601000174, 660230002..."
2,304576,"[6602300021, 6601000174, 6601000045, 100000102..."
3,304576,"[6602300021, 6601000174, 6601000045, 100000102..."
4,304576,"[6601000174, 10000010251, 10000009271, 6602300..."
5,304576,"[6601000174, 10000010251, 10000009271, 6602300..."
6,304576,"[10000010251, 10000009271, 10000007607, 660100..."
7,304576,"[6601000174, 10000009978, 10000009271, 6601000..."
8,304576,"[6601000174, 6601000043, 10000010251, 66010000..."
9,304576,"[6601000174, 6601000043, 10000010251, 66010000..."


In [33]:
# y아이템은 row단위로
y_ProductCodes = []
for ii in df2_2.y_ProductCodes.index:
    for jj in df2_2.y_ProductCodes[ii]:
        y_ProductCodes.append((ii, jj))
df3_2 = pd.DataFrame(y_ProductCodes, columns=[ 'index','y_ProductCodes'] ).set_index('index')

In [34]:
df3_2

,y_ProductCodes
index,
0,6602300021
0,6601000174
0,6601000045
0,10000010251
0,10000009978
0,6601000093
0,6601000041
1,10000009271
1,6600200174


In [35]:
# 모든 정보 결합
df4 = pd.merge(
    pd.concat([df3_1,
#                x_d2v_embed,
               df3_2],axis=1), 
    df2[['BpID','Region','ConstructionType','BpType','BpClass']].drop_duplicates(), 
    on='BpID', how='left')
print(df4.shape)

(424605, 7)


In [36]:
df4.head()

,BpID,x_ProductCodes,y_ProductCodes,Region,ConstructionType,BpType,BpClass
0,304576,"[6601000050, 6601000055, 6601000174, 100000080...",6602300021,12,7,161,96
1,304576,"[6601000050, 6601000055, 6601000174, 100000080...",6601000174,12,7,161,96
2,304576,"[6601000050, 6601000055, 6601000174, 100000080...",6601000045,12,7,161,96
3,304576,"[6601000050, 6601000055, 6601000174, 100000080...",10000010251,12,7,161,96
4,304576,"[6601000050, 6601000055, 6601000174, 100000080...",10000009978,12,7,161,96


In [37]:
df2_2.head()

,BpID,x_ProductCodes,y_ProductCodes
0,304576,"[6601000050, 6601000055, 6601000174, 100000080...","[6602300021, 6601000174, 6601000045, 100000102..."
1,304576,"[6601000050, 6601000055, 6601000174, 660230002...","[10000009271, 6600200174, 7400900162]"
2,304576,"[6602300021, 6601000174, 6601000045, 100000102...","[6601000174, 6602300001, 6601000067, 100000076..."
3,304576,"[6602300021, 6601000174, 6601000045, 100000102...",[10000010251]
4,304576,"[6601000174, 10000010251, 10000009271, 6602300...","[6601000060, 10000009271, 10000007607]"


In [38]:
df4.head()

,BpID,x_ProductCodes,y_ProductCodes,Region,ConstructionType,BpType,BpClass
0,304576,"[6601000050, 6601000055, 6601000174, 100000080...",6602300021,12,7,161,96
1,304576,"[6601000050, 6601000055, 6601000174, 100000080...",6601000174,12,7,161,96
2,304576,"[6601000050, 6601000055, 6601000174, 100000080...",6601000045,12,7,161,96
3,304576,"[6601000050, 6601000055, 6601000174, 100000080...",10000010251,12,7,161,96
4,304576,"[6601000050, 6601000055, 6601000174, 100000080...",10000009978,12,7,161,96


In [39]:
model_dir = './Trained_models/Models_WnD/rrr/'

VocaList = []
for ii in df2_2.x_ProductCodes.index:
    for jj in df2_2.x_ProductCodes[ii]:
        VocaList.append(int(jj))
VocaList = list(set(VocaList))

ConstructList = list(df4.ConstructionType.unique())
RegionList = list(df4.Region.unique())
BpTypeList = list(df4.BpType.unique())
BpClassList = list(df4.BpClass.unique())
LabelList = tuple(df4.y_ProductCodes.unique())

In [40]:
VocaList

[10000039940,
 2801300093,
 10000039945,
 2801300095,
 10000039970,
 10000039971,
 10000025096,
 10000039973,
 10000039974,
 10000039975,
 10000039972,
 10000039977,
 10000039980,
 2801300101,
 10000039983,
 10000039985,
 10000039990,
 10000040000,
 2801300108,
 10000007254,
 10000040031,
 10000007264,
 10000007283,
 10000007354,
 10000007355,
 10000007357,
 10000007358,
 10000009467,
 10000007363,
 10000007364,
 10000007365,
 10000007366,
 10000040135,
 10000007369,
 10000007370,
 10000007371,
 10000007372,
 10000040137,
 10000040142,
 10000040147,
 10000040148,
 10000040149,
 10000040150,
 10000040151,
 10000007384,
 10000040152,
 10000040154,
 10000040155,
 10000007388,
 10000040157,
 10000040153,
 10000007391,
 10000040159,
 10000040160,
 10000040162,
 10000007397,
 10000040167,
 10000007402,
 2800600106,
 10000007405,
 10000007407,
 10000007408,
 10000007409,
 10000007410,
 10000007411,
 10000007414,
 10000007415,
 2801300141,
 10000007417,
 10000007418,
 10000007419,
 10000007420

In [42]:
def make_Example(row):
    example= tf.train.Example(features=tf.train.Features(feature={
#         'BpID': tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['BpID'].encode('utf-8')])),
        'Region': tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Region'].encode('utf-8')])),
        'BpType': tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['BpType'].encode('utf-8')])),
        'BpClass': tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['BpClass'].encode('utf-8')])),
        'ConstructionType': tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['ConstructionType'].encode('utf-8')])),
        'x_ProductCodes': tf.train.Feature(int64_list=tf.train.Int64List(value= row['x_ProductCodes'])),
        'y_ProductCodes': tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['y_ProductCodes'].encode('utf-8')]))
    }))
    return example

def create_tfrecords_file(tf_filename= 'data.tfrecord', input_df=None):
    writer = tf.python_io.TFRecordWriter(tf_filename)
    print("Creating TFRecords file at", tf_filename, "...")
    def mk_tfrecords(row):
        example = make_Example(row)
        content = example.SerializeToString()
        writer.write(content)
        return example
    input_df.apply(mk_tfrecords, axis=1)
    writer.close()
    print("Finish Writing", tf_filename)

In [ ]:
input_df = df4.drop(['BpID'],axis=1)

input_train, input_test = train_test_split(input_df.values, test_size=0.2)

create_tfrecords_file('./Datasets/train.tfrecord', pd.DataFrame(input_train, columns=input_df.columns))
create_tfrecords_file('./Datasets/eval.tfrecord', pd.DataFrame(input_test, columns=input_df.columns))

In [43]:
# Parsing features
read_features = {
#         'BpID': tf.FixedLenFeature([], dtype=tf.string),
        'Region': tf.FixedLenFeature([], dtype=tf.string),
        'BpType': tf.FixedLenFeature([], dtype=tf.string),
        'BpClass': tf.FixedLenFeature([], dtype=tf.string),
        'ConstructionType': tf.FixedLenFeature([], dtype=tf.string),
        'x_ProductCodes': tf.VarLenFeature(dtype=tf.int64),
        'y_ProductCodes': tf.FixedLenFeature([], dtype=tf.string)
}

ERROR! Session/line number was not unique in database. History logging moved to new session 687


In [44]:
def input_fn(filename):
    def parser(serialized_example):
        columns = tf.parse_single_example(
            serialized=serialized_example, 
            features=read_features)
        label = columns['y_ProductCodes']
        del columns['y_ProductCodes']
        features = columns
        return features, label
    
    dataset= tf.data.TFRecordDataset(filenames=filename)
    dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(buffer_size = 100, count = int(5e5)))

    dataset = dataset.apply(tf.contrib.data.map_and_batch(
            map_func=parser, batch_size=64, num_parallel_calls = 16))

    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size = 50)
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

In [45]:
def train_input_fn():
    return input_fn(filename='./Datasets/train.tfrecord')

def eval_input_fn():
    return input_fn(filename='./Datasets/eval.tfrecord')

In [46]:
BpClass=tf.contrib.layers.sparse_column_with_hash_bucket("BpClass", hash_bucket_size=300)
BpType=tf.contrib.layers.sparse_column_with_hash_bucket("BpType", hash_bucket_size=300)
ConstructionType=tf.contrib.layers.sparse_column_with_hash_bucket("ConstructionType", hash_bucket_size=100)
Region=tf.contrib.layers.sparse_column_with_hash_bucket("Region", hash_bucket_size=100)
X_item = tf.feature_column.categorical_column_with_vocabulary_list("x_ProductCodes",vocabulary_list=VocaList)

C:\Users\aj901\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
C:\Users\aj901\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\aj901\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [48]:
wide_columns = [
    X_item,
    ConstructionType,
#     tf.feature_column.crossed_column([X_item, X_item], hash_bucket_size=int(1e7)),
    tf.contrib.layers.crossed_column([ConstructionType, Region], hash_bucket_size=int(1e3)),
    tf.contrib.layers.crossed_column([BpClass, BpType], hash_bucket_size=int(1e5))
]
deep_columns = [
    tf.contrib.layers.embedding_column(BpClass, dimension=8), 
    tf.contrib.layers.embedding_column(BpType, dimension=8), 
    tf.contrib.layers.embedding_column(ConstructionType, dimension=8), 
    tf.contrib.layers.embedding_column(Region, dimension=8)
]

C:\Users\aj901\Anaconda3\lib\site-packages\tensorflow\python\platform\tf_logging.py:120: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  _get_logger().warn(msg, *args, **kwargs)


In [49]:
# model config setup for speed up (reference : tensorflow performance guide)
# config = tf.ConfigProto()
# config.intra_op_parallelism_threads = 256 # 8->16->32->64....
# config.inter_op_parallelism_threads = 0 # 0으로 고정
_config = tf.contrib.learn.RunConfig(
#     session_config=config, 
    save_checkpoints_secs=None, 
    save_checkpoints_steps=1000,
    keep_checkpoint_max=2)

tot_model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir=model_dir, 
    linear_feature_columns=wide_columns, 
    dnn_feature_columns=deep_columns, 
    dnn_hidden_units= [1024, 512, 256], 
    dnn_dropout = 0.5, 
    n_classes = len(LabelList), 
    label_vocabulary=LabelList, 
    batch_norm = True,
    config = _config)

# https://www.tensorflow.org/api_docs/python/tf/estimator/train_and_evaluate
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)
tf.estimator.train_and_evaluate(tot_model, train_spec, eval_spec)
# tot_model.train(input_fn=train_input_fn,max_steps=100000)

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002536349EF28>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_protocol': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': 1000, '_keep_checkpoint_max': 2, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './Trained_models/Models_WnD/rrr/'}
INFO:tensorflow:Not using Distribute Coor

C:\Users\aj901\Anaconda3\lib\site-packages\tensorflow\python\util\tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always use FingerprintCat64
to concatenate the feature fingerprints. And the underlying
_sparse_feature_cross_op.sparse_feature_cross operation will be marked
as deprecated.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./Trained_models/Models_WnD/rrr/model.ckpt.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-12-06:19:56
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Trained_models/Models_WnD/rrr/model.ckpt-0


OutOfRangeError: Read fewer bytes than requested
	 [[node save/RestoreV2_1 (defined at <ipython-input-49-cca55ae0f206>:25)  = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2_1/tensor_names, save/RestoreV2_1/shape_and_slices)]]

Caused by op 'save/RestoreV2_1', defined at:
  File "C:\Users\aj901\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\aj901\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\aj901\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\aj901\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\aj901\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
   

In [ ]:
# tot_model.evaluate(input_fn=eval_input_fn)

In [32]:
# # # 커스텀 평가기준
# custom_evaluation()

In [ ]:
#https://www.tensorflow.org/performance/datasets_performance
#https://www.tensorflow.org/performance/performance_guide
#https://www.tensorflow.org/performance/performance_models
"""
181109 테스트로 돌려본 모델
181108 테스트로 돌려본 모델에서...
◆ 와이드 카테고리 컬럼 6가지 모든 조합 크로스 컬럼 추가.
◎ 
################################################
181108 퇴근시 돌리고 간 모델
181107 퇴근시 돌리고 간 모델에서...
◆ 기본적으로 10/31일 스크립트에서 가져옴. 아래는 변동사항.
◆ 아담 옵티마이저 적용
◆ (와이드, 크로스)사업유형, 사업클래스 포함
◆ (딥)모든 임베딩 128차원으로 변경
◆ 드랍아웃을 0.65로 변경.(입력은0.35)
◎ 성공. 100분당 6분으로 단축.
################################################
181107 퇴근시 돌리고 간 모델
181106 퇴근시 돌리고 간 모델에서...
◆ (와이드,딥)배송권역 제외하기
◆ (와이드)사업유형, 사업클래스 제외하기
◆ (와이드, 크로스)기간 제외하기
◆ 히든유닛 [2048,1024, 512, 256]->[500,200]
◆ BpClass, BpType : 3 // ConstructionType, Region : 2 차원으로 Embedding_column에서 변경.
◎ 결과는 엑셀 참고할것.
################################################
181106 퇴근시 돌리고 간 모델
181106 테스트로 돌려본 모델에서, 변동없음.
◎ 성공 15000회, 100회당 7분소요. loss:train : 확인불가.
◎ 커스텀 평가결과 [3]:[0.05], [5]:[0.13], [10]:[0.4]개 구매함. 11/5일 기록보다 성적 저조.
################################################
181106 테스트로 돌려본 모델
181105 퇴근시 돌리고 간 모델에서,
◆ 기간을 추가. 기간은 Y일자-각각의 X일자의 제곱합. 버켓타이즈하여 와이드, 딥, 크로스에 반영.
◆ 와이드 크로스에서 사업장유형, 클래스는 제거. 나머지는 유지
◆ 딥유닛수 : [1024, 512, 256]->[2048,1024, 512, 256]
◎ 성공.
################################################
181105 퇴근시 돌리고 간 모델.
181102 퇴근시 돌리고 간 모델에서 수정없이 훈련 시도.
◆ 배송권역을 개별 텐서로 따로 처리. + 딥과 크로스에 추가.
◆ 딥유닛수 : [2000,1000,500]->[1024, 512, 256], buffer_size = 100
◎ 성공 - 16000회, 100회당 7분소요. loss:train=5.4/eval=5.9까지 도달.
◎ 커스텀 평가결과 [3]:[0.10], [5]:[0.23], [10]:[0.83]개 구매함.성적 매우 저조.
################################################
181102 퇴근시 돌리고 간 모델
181101 퇴근시 돌리고 간 모델에서 아래 부분만 수정해서 테스트 함
◆ 배송권역을 개별 텐서로 따로 처리. + 딥과 크로스에 추가.
◆ 딥유닛수 : [2000,1000,500]->[1024, 512, 256], buffer_size = 100
◎ 주말에 서버 재부팅 한다하여 실행하지 않음. 
################################################
181101 퇴근시 돌리고 간 모델
181031 퇴근시 돌리고 간 모델에서 아래 부분만 수정해서 테스트 함
◆50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
◆배송권역을 추가함(세션레벨 -> 멀티핫 처리) -> CSV파일들도 재생성.
◆배송권역은, 와이드에만 아이템과 묶어서 추가.
◆쓰레드 128,0->256,0, 넘패러럴 16, 딥유닛수 [500,200]->[2000,1000,500], 체크포인트맥스 5->2
◎ 성공, 15000회, 100회당 7분 소요. loss:train=Unknown/eval=6.1까지 도달.
◎ 커스텀 평가결과 [3]:[0.16], [5]:[0.41], [10]:[1.41]개 구매함. 11/1일 기록보다 성적 저조.
################################################
181031 퇴근시 돌리고 간 모델
181031 테스트로 돌려본 모델1에서 아래 부분만 수정해서 테스트 함
◆50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
◆구매자 특징 4종을 와이드에 추가함
◆구매자 특징 4종을 2종/2종끼리 묶어 크로스 컬럼으로 와이드에 추가함.
◆쓰레드 128,0
◎ 성공, 13000회, 100회당 7분30초 소요. loss:train=5.3/eval=6.5까지 도달. 특이사항 : 모델용량이 엄청 커짐.
◎ 커스텀 평가결과 [3]:[0.26], [5]:[0.63], [10]:[2.16]개 구매함. 
################################################
181031 테스트로 돌려본 모델1
181030 퇴근시 돌리고 간 모델에서 아래 부분만 수정해서 테스트 함
◆50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
◆하이레벨 텐서 셋팅을 기존 튜토리얼대로 다시 재구성함.
◆parse_csv부분도 튜토리얼에서 재검증하여 재구성함.
◆쓰레드 64,0
◎텐서보드 확인결과 dnn 0에서 움직이지 않음. 그러나 움직이지 않은 원인 파악 성공. 앞으로는 무시
################################################
181030 퇴근시 돌리고 간 모델
181029 퇴근시 돌리고 간 모델에서 아래 부분만 수정해서 테스트 함
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
D2V컬럼을 tf.string_split->SparseTensor로 변경.
doc2vec 컬럼 모두 비활성화후 훈련시킴.
성공, 14000회, 100회당 7분 소요. loss:train=5.9/eval=6.2까지 도달, 텐서보드 확인결과 dnn 0에서 움직이지 않음.
################################################
181030 테스트로 돌려본 모델2
181029 퇴근시 돌리고 간 모델에서 아래 부분만 수정해서 테스트 함
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
D2V컬럼을 tf.string_split->SparseTensor로 변경.
카테고리 컬럼 모두 비활성화후 훈련시킴.
성공, 12000회, 100회당 ?분 소요. 로스트 ?까지 도달, 텐서보드 확인결과 dnn 0에서 움직이지 않음
################################################
181030 테스트로 돌려본 모델1
181029 퇴근시 돌리고 간 모델에서 아래 부분만 수정해서 테스트 함
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
D2V컬럼을 tf.string_split->SparseTensor로 변경.
텐서보드 확인결과 dnn 0에서 움직이지 않음
################################################
181029 퇴근시 돌리고 간 모델...
181026 퇴근시 돌리고 간 모델에서 아래 부분만 수정해서 테스트 함
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
D2V컬럼을 reshape하고 훈련함.
성공, 12000회, 100회당 7분 소요. loss:train=5.5/eval=6.8까지 도달, 텐서보드 확인결과 dnn 0에서 움직이지 않음.
################################################
181026 퇴근시 돌리고 간 모델...**********************************************************
181026 테스트로 돌려본 모델1을 기반으로 셋팅을 확장해서 테스트함.
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
num_parallel_calls = 16, 쓰레드 32,0, 액티베이션 = elu, 히든레이어 1000추가, 옵티마이저 =아담, 러닝레이트 0.001
성공, 56000회(금요일 6시 ~ 월요일 9시), 100회당 7분소요. 로스틑 6.1까지 도달. 
################################################
181026 테스트로 돌려본 모델6
181026 테스트로 돌려본 모델1을 기반으로 셋팅을 확장해서 테스트함.
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
num_parallel_calls = 16, 쓰레드 32,0, 액티베이션 = elu, 선형모델 부분은 None 처리하고 DNN만 돌려본다.
최적화 되는데?
################################################
181026 테스트로 돌려본 모델5
181026 테스트로 돌려본 모델1을 기반으로 셋팅을 확장해서 테스트함.
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
num_parallel_calls = 16, 쓰레드 32,0, 액티베이션 = elu
텐서보드상에서 DNN은 계속 0인 상태임.
################################################
181026 테스트로 돌려본 모델4
181026 테스트로 돌려본 모델1을 기반으로 셋팅을 확장해서 테스트함.
50, 200, 10 셋팅 -> 아이템 : 3375, 크로스 : 3543, 총 : 7223
100회에 430초 소요
대충 7500~ 8000사이까지가 현재상태에서는 컬럼수 최대치인듯.
################################################
181026 테스트로 돌려본 모델3
181026 테스트로 돌려본 모델1을 기반으로 셋팅을 확장해서 테스트함.
30, 200, 10 셋팅 -> 아이템 : 4421, 크로스 : 3543, 총 : 8269
num_parallel_calls = 16, 쓰레드 8,0
뻑남.... 
################################################
181026 테스트로 돌려본 모델2
181026 테스트로 돌려본 모델1을 기반으로 셋팅을 확장해서 테스트함.
30, 100, 10 셋팅 -> 아이템 : 4421, 크로스 : 10646, 총 : 15372
num_parallel_calls = 16, 쓰레드 0,0
뻑남....
################################################
181026 테스트로 돌려본 모델1
아이템과 크로스 컬럼에 해당하는 tensor들을 input_fn에서 stack해 주고 2차원 매트릭스 형태로
real-valued column -> bucketized column에 넣어줌.
doc2vec 300차원을 1개의 high level tensor로 변형한건 유지
100, 300, 10 셋팅 -> 아이템 : 2111, 크로스 : 1848, 총 : 4264
num_parallel_calls = 16, 쓰레드 256, 256. -> 쓰레드 0,0
속도가 엄청나게 빨라진다.... 특징 컬럼의 수가 증가함에 따라 속도가 증가한다는걸 깨달음.
Cross -> TF Crosscolumn 사용 시도하였으나 feature수의 폭증으로 취소
-> decode_csv에서 100차원을 표현하기 위해 100개의 tensor가 자동으로 만들어지기 때문에 속았다!
-> 100차원을 표현하기위해 100개의 tensor를 만들기 보다 100차원의 1개 텐서를 만드는게 옳다.
성공
################################################
181025 퇴근시 돌리고 간 모델...
100, 300, 10 셋팅 -> 아이템 : 2111, 크로스 : 1848, 총 : 4264
num_parallel_calls = 16, 쓰레드 256, 256.
doc2vec 300차원을 1개의 high level tensor로 변형하여 트레이닝 함.
6000회 훈련에 약 14시간 소요되었고 
train-loss : 7.0 / eval-loss : 7.8
좀더 loss값 감소의 여지는 있으나 거의 수렴.
################################################
181025 테스트로 돌려본 모델2
50,100,10 셋팅 -> 아이템 : 3375, 크로스 : 10646, 총 : 14326
num_parallel_calls = 16, 쓰레드 256, 256.
NFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./Trained_models/Models_WnD/StartDate_181025/model.ckpt.
INFO:tensorflow:loss = 545.25824, step = 1
INFO:tensorflow:global_step/sec: 0.0117695
INFO:tensorflow:loss = 10360.906, step = 101 (8496.533 sec) # 처음 100회에 2시간 반 소요..........
이후 커널뻑남.....
################################################
181025 테스트로 돌려본 모델(간략버전)
파일 불러오는 스크립트 오류. 수정완료.
num_parallel_calls = 8, 쓰레드 128, 128.
doc2vec 300차원을 1개의 high level tensor로 변형하여 트레이닝 함.
테스트 성공
################################################
181024 퇴근시 돌리고 간 모델...
100, 300, 10 셋팅 -> 아이템 : 2111, 크로스 : 1848, 총 : 4264
num_parallel_calls = 8, 쓰레드 128, 128.
doc2vec 300차원을 1개의 high level tensor로 변형하여 트레이닝 함.
작동하지 않음. 원인은? 300차원을 1개의 high level tensor에 넣어서 텐서가 2GB 용량 제한에 걸린건가?
22일 기준으로 원복.
################################################
181024 업무시간에 돌려본 모델...
100, 300, 10 셋팅
num_parallel_calls = 2 -> 8로 변경, 쓰레드 256, 256 적용.
이전 모델보다 훨씬 느리다. 다른 작업들이 수행중인게 있어서 그런 것인가?
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./Trained_models/Models_WnD/StartDate_181024/model.ckpt.
INFO:tensorflow:loss = 540.76764, step = 1
INFO:tensorflow:global_step/sec: 0.0727103
INFO:tensorflow:loss = 1890.2057, step = 101 (1375.334 sec)
INFO:tensorflow:global_step/sec: 0.129333
INFO:tensorflow:loss = 1187.8994, step = 201 (773.200 sec)
################################################
181023 퇴근시 돌리고 간 모델...
100, 300, 10 셋팅
인풋은 넘파이에서 바로(num_parallel_calls 옵션 없음.)
쓰레드 128, 128 적용.
작동 하지 않음.
################################################
181022 퇴근시 돌리고 간 모델...
100, 300, 10 셋팅
num_parallel_calls = 2 -> 8로 변경, 배치 100->64로 변경, 쓰레드 64, 64 적용.
100 iter 당 10분 소요.
9000회 훈련에 약 14시간 소요되었고 
train-loss : 6.7 / eval-loss : 7.6
거의 수렴함.
################################################
20181022 16:00 작업시작~15:00 작업종료건. 100, 300, 10 셋팅
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./Trained_models/Models_WnD/StartDate_181022/model.ckpt.
INFO:tensorflow:loss = 853.93884, step = 1
INFO:tensorflow:global_step/sec: 0.0862069
INFO:tensorflow:loss = 3197.085, step = 101 (1160.012 sec)
INFO:tensorflow:global_step/sec: 0.158939
INFO:tensorflow:loss = 1942.4188, step = 201 (629.164 sec)
INFO:tensorflow:global_step/sec: 0.156028
INFO:tensorflow:loss = 1513.222, step = 301 (640.913 sec)
INFO:tensorflow:global_step/sec: 0.153983
INFO:tensorflow:loss = 1389.647, step = 401 (649.430 sec)
"""

# 181025일 기준
# D2V -> 1 하이레벨텐서로 묶음.

In [50]:
def input_fn(data_files, num_epochs, batch_size):
    def parse_csv(value):
        columns = tf.decode_csv(value, record_defaults=iK['_csv_col_defaults'])
        feature_ex_d2v = columns[:-301] #레이블 포함 마지막 301개 컬럼은 항상 d2v + 1label여야 한다.
        tmp_zip1 = dict(zip(iK['featurenm'][:-300], feature_ex_d2v))
        feature_d2v = columns[-301:-1] # 마지막 300개 features = d2v
        tmp_zip2 = {'d2v': tf.stack(feature_d2v, axis = -1)}
        label = columns[-1]
        d = dict(**tmp_zip1, **tmp_zip2), label
        return d
    
    dataset = tf.data.TextLineDataset(data_files)
    dataset = dataset.apply(tf.contrib.data.shuffle_and_repeat(buffer_size = 400000, count = num_epochs))
    dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=parse_csv, 
                                                          batch_size=batch_size, 
                                                          num_parallel_calls = 16))
    dataset = dataset.cache()
    dataset = dataset.prefetch(buffer_size = 50)
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

In [51]:
filenames = ["./Datasets/"+f_name for f_name in os.listdir('./Datasets/') if f_name.startswith('dataset') and f_name.endswith('.csv')]
filename_split = train_eval_filesplit(filenames = filenames, train_rate=0.8)

def train_input_fn():
    return input_fn(data_files=filename_split['train_files'], num_epochs=int(1e6), batch_size=64)

def eval_input_fn():
    return input_fn(data_files=filename_split['eval_files'], num_epochs=1, batch_size=1)

print(filename_split)

FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './Datasets/'

In [ ]:
# item + 크로스 컬럼을 아래와 같이?
# test = tf.contrib.layers.real_valued_column(column_name=test, dimension=???, dtype=tf.int64)
# test_buckets = tf.contrib.layers.bucketized_column(test, boundaries=[1])
# 또는 위를 아이템만 하고
# tf.contrib.layers.crossed_column([test_buckets, test_buckets], hash_bucket_size=?1??int(1e4))

def TensorGenerator(w_co_item, w_co_cros, w_ca_else, d_co_d2v, d_ca_else):
    wide_cont_item=[tf.contrib.layers.sparse_column_with_integerized_feature(column_name=ii, bucket_size=1) for ii in w_co_item]
    wide_cont_cros=[tf.contrib.layers.sparse_column_with_integerized_feature(column_name=jj, bucket_size=1) for jj in w_co_cros]
    wide_cate_else=tf.contrib.layers.sparse_column_with_keys(column_name=w_ca_else, keys=iK[w_ca_else])
    
    deep_cont_d2v =tf.contrib.layers.real_valued_column(column_name=d_co_d2v, dimension=300, dtype=tf.float64)
    deep_cate_else=[tf.contrib.layers.sparse_column_with_keys(column_name=ll, keys=iK[ll]) for ll in d_ca_else]
    deep_cate_else=[tf.contrib.layers.embedding_column(
            sparse_id_column = deep_cate_else[mm], 
            dimension = len(iK[d_ca_else[mm]])) for mm in range(len(d_ca_else))]
    
    wide_columns = wide_cont_item + wide_cont_cros + [wide_cate_else]
    deep_columns = [deep_cont_d2v] + deep_cate_else
    return {'wide_columns' : wide_columns, 'deep_columns' : deep_columns}

In [ ]:
TensorList = TensorGenerator(
    w_co_item = iK['x_item_colnm'], 
    w_co_cros = iK['x_cross_colnm'],
    w_ca_else = 'ConstructionType',
    d_co_d2v = 'd2v', 
    d_ca_else = iK['x_else_colnm'])

In [ ]:
# model config setup for speed up (reference : tensorflow performance guide)
config = tf.ConfigProto()
config.intra_op_parallelism_threads = 256# 256->512->1024
config.inter_op_parallelism_threads = 256# 256->512->1024
_config = tf.contrib.learn.RunConfig(
    session_config=config, 
    save_checkpoints_secs=None, 
    save_checkpoints_steps=1000)

In [ ]:
tot_model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir=model_dir, 
    linear_feature_columns=TensorList['wide_columns'],
    dnn_feature_columns=TensorList['deep_columns'],
    dnn_hidden_units= [500, 200], dnn_dropout = 0.5, 
    n_classes = len(iK['y_classes']), label_vocabulary=iK['y_classes'], batch_norm = True,
    config = _config
)

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/estimator/train_and_evaluate
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)
tf.estimator.train_and_evaluate(tot_model, train_spec, eval_spec)
# 30,100,0은 뻑남. ---소요시간 반나절.
# 50,100,10은 뻑남 ---소요시간 하루종일 -> 100번까지 돌다가 뻑남.
# 100,300,10은 --- 돌아간다.

In [ ]:
# #플레이스 홀더 초기화 포함한 코드... 점검 안해봄.

# # https://stackoverflow.com/questions/45620449/initialization-of-tf-contrib-data-iterator-with-tf-estimator
# class IteratorInitHook(tf.train.SessionRunHook):
#     def after_create_session(self, session, coord):
#         session.run(self.iterator_init_op)

# def train_input_fn():
#     init_hook = IteratorInitHook()
#     _input = input_fn(X=X_train, y=y_train, num_epochs=int(1e6), batch_size=64, buffer_size=len(X_train))
#     return _input, init_hook

# def eval_input_fn():
#     init_hook = IteratorInitHook()
#     _input = input_fn(X=X_test, y=y_test, num_epochs=1, batch_size=64, buffer_size=len(X_test))
#     return _input, init_hook

# train_input_fn, train_init_hook = train_input_fn()
# test_input_fn, test_init_hook = eval_input_fn()

# train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, hooks = train_init_hook)
# eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, hooks = test_init_hook)
# tf.estimator.train_and_evaluate(tot_model, train_spec, eval_spec)

In [ ]:
# # 데일리 예측 결과 뽑기용
# prediction = tot_model.predict(input_fn=eval_input_fn)
# prediction_result = []
# for num, item in enumerate(prediction):
#     if num % 10 ==0:
#         print(num)
#     if num >= 100: #eval01.shape[0]:
#         break
#     else:
#         prediction_result.append(item['probabilities'])

In [ ]:
# # 평가용 - 구매아이템 A가 추천 아이템 3  or 5 or 10개중에 포함될 확률
# eval_threshold = 10
# eval01 = np.loadtxt(fname="./Datasets/dataset4.csv",delimiter=',',dtype='str')
# eval02 = pd.Series(iK['y_classes'])

# # 예측 결과 출력
# prediction = tot_model.predict(input_fn=eval_input_fn)
# result = []
# for num, item in enumerate(prediction):
#     if num >= 100: #eval01.shape[0]:
#         break
#     else:
#         rateeval1 = pd.Series(item['probabilities']).sort_values(ascending = False).iloc[:eval_threshold] # 가장높은 확률 K개
#         rateeval2 = eval02[eval02.index.isin(rateeval1.index)].values # 가장높은 확률에 해당하는 아이템 리스트업
#         result.append(int(eval01[:,-1][ii] in rateeval2)) # 정답과 비교
#         print(result[-1])

In [ ]:





#### 과거 스트립트








In [168]:
# Prediction format
# 181012
#이전 n개의 기록으로 k개의 장바구니를 예측(k와n 사이 최소기간 p-day
"""
일 기준으로 리스트 업 하려면 아래 코드 사용.
df1['OrderTime'] = df1['OrderTime'].dt.date
df02 = df1.groupby(['BpID','OrderTime']).agg({
        'OrderNum': lambda x: list(x), 
        'ProductCode' : lambda x: list(x)}).reset_index()
오전/오후 구분하여 적용하려면 아래 코드 사용하여 groupby
df1['afternoonTF'] = (df1['OrderTime'].dt.time > datetime.time(12)).astype('int')
주문건 기준으로 리스트업 하려면 아래 코드 사용
df1 = df[['BpID','OrderTime','OrderNum','ProductCode']].drop_duplicates(keep='first')
df1['OrderTime'] = pd.to_datetime(df1['OrderTime'])
df1 = df1.sort_values(['BpID','OrderTime']).reset_index(drop = True)
df2 = df1.groupby(['BpID','OrderTime','OrderNum'])['ProductCode'].apply(list).reset_index()
        
"""
nx =  3 # 이전 기록 갯수(X)
ny = 1 # 이후 기록 갯수(Y)
df2_1 = {
    'BpID':[], 'OrderTime':[],
    'x_OrderNums':[], 'y_OrderNums':[],
    'x_ProductCodes':[], 'y_ProductCodes':[]
}
print('Loop start!!')
start = datetime.datetime.now()
for ii in range(len(df2)-nx):
    targetdata = df2.iloc[ii:ii+nx+ny,].reset_index(drop=True)
    if len(targetdata['BpID'].unique())==1 and len(targetdata) == nx+ny: # 앞 nx+ny 가 모두 같은 사업장이고 사이즈가 nx+ny인 경우
        tmpdata1 = targetdata['OrderNum'].tolist()
        tmpdata2 = targetdata['ProductCode'].tolist()
        df2_1['BpID'].append(targetdata['BpID'][0])
        df2_1['OrderTime'].append(targetdata['OrderTime'][nx-ny])
        df2_1['x_OrderNums'].append([','.join(tmpdata1[0:nx])])
        df2_1['y_OrderNums'].append([','.join(tmpdata1[nx:nx+ny])])
        df2_1['x_ProductCodes'].append([','.join(list(chain.from_iterable(tmpdata2[0:nx])))])
        df2_1['y_ProductCodes'].append([','.join(list(chain.from_iterable(tmpdata2[nx:nx+ny])))])
    if ii % 10000 == 0:
        duration = datetime.datetime.now()-start;m, s = divmod(duration.seconds, 60)
        h, m = divmod(m, 60);print("[%d]/[%d]-----[%02d:%02d:%02d]" %(ii, len(df2)-nx, h, m, s)) 
df3 = pd.DataFrame(df2_1)

In [ ]:
# 아이템 임베딩
# 데이터를 one-hot 임베딩 작업 -- 181010
# dataset_X
X_data['x_items'] = [ii.split(',') for ii in X_data.tmp_x_items]
mlb = MultiLabelBinarizer()
x_item_embed = pd.DataFrame(mlb.fit_transform(X_data['x_items']), columns=mlb.classes_, index=X_data['x_items'].index)
x_item_embed.columns = ["x_" + ii for ii in x_item_embed.columns.tolist()]

In [ ]:
# 이하 데이터 불필요건 제거 작업
set_size = 1000 # n개씩 끊어서 처리
print_size = 50 # 진행과정 확인 주기
threshold = 30 # n회 미만 구매이력 있는 아이템의 경우는 제거하자
data_len = int(x_item_embed.shape[0]/set_size)+2

# 한방에는 너무 오래걸리니 나눠서 한다.
tmp02 = np.zeros(x_item_embed.shape[1])
for ii in range(data_len):
    if ii % print_size ==0:
        print(ii, data_len)
    tmp01 = x_item_embed.iloc[ii*set_size:ii*set_size+set_size,].values.sum(axis=0)
    tmp02 += tmp01
tmp03 = tmp02 > threshold
%time x_item_embed_1 = x_item_embed.loc[:,tmp03]
print(x_item_embed.shape, '-->', x_item_embed_1.shape)

In [ ]:
# n회 미만 구매이력 있는 아이템을 제거하면 구매이력이 없어지는 데이터를 X,y에서 제거하자.
drop_target_index = x_item_embed_1[x_item_embed_1.sum(axis = 1) ==0].index

X_data = X_data.drop(drop_target_index, axis = 0)
x_item_embed_1 = x_item_embed_1.drop(drop_target_index, axis = 0)
y_item_embed = y_item_embed.drop(drop_target_index, axis = 0)
x_cross_embed = x_cross_embed.drop(drop_target_index, axis = 0)
print('x_item:', x_item_embed_1.shape,  '\n', 'x_cross:', x_cross_embed.shape, '\n', 'y_item:', y_item_embed.shape)

In [ ]:
y_list1 = [ii.split(',') for ii in X_data.tmp_y_items]

aa = []
bb = []
for ii, jj in pd.Series(y_list1).iteritems():
    aa.extend(np.repeat(ii,len(jj)))
    bb.extend(jj)
    
y_df = pd.DataFrame({'index': aa, 'y_items' : bb})

In [ ]:
# from imblearn.over_sampling import RandomOverSampler # 복잡한 샘플링은 메모리 부족ㅠㅠ

# even_target1 = model_df03[model_df03.ConstructionType.isin(['SKT 전송선로', 'SKB 1군', 'SKT A망 1군'])]
# pre_y = even_target1.ConstructionType
# pre_X = even_target1.drop(['ConstructionType'],axis = 1)
# y = pre_y.tolist()
# X = pre_X.values.tolist()

# ros = RandomOverSampler(random_state=0)
# X_resampled, y_resampled = ros.fit_sample(X, y)

# print("asis : [%s] -->\n tobe : [%s]" %(sorted(Counter(y).items()), sorted(Counter(y_resampled).items())))

# even_data_X = pd.DataFrame(X_resampled, columns= pre_X.columns)
# even_data_y = pd.DataFrame({'ConstructionType' : y_resampled})
# even_data = pd.merge(even_data_X, even_data_y, left_index=True, right_index=True)

# ex_index = set(model_df03.index) - set(even_target1.index)
# model_df03 = pd.concat([even_data, model_df03.loc[ex_index,:]], sort= True)
# model_df03.shape

In [2]:
def Doc2Vec(data, d2v_size, model_loc, epoch, trainTF = False):
    """
    data : DataFrame([['DocID','Item']])
    """
    data.columns = ['docID','item']
    data1 = [(str(row['item']).split(), row['docID']) for idx, row in data.iterrows()]
    data2 = namedtuple('TaggedDocument', 'words tags')
    tagged_data2 = [data2(d, [c]) for d, c in data1]
    
    if trainTF:
        model = Doc2Vec(
            dm = 0,  # 0 : PV-DBOW
            dbow_words = 0,  # 0 : train doc-vec only(faster)
            window = 8, vector_size = d2v_size, alpha = 0.025, min_alpha = 0.025, seed = 0, sample= 1e-5, min_count=3, 
            workers=multiprocessing.cpu_count(), hs = 0, negative = 10)
        model.build_vocab(tagged_data2)
        print('New model training started')
        model.train(documents  = tagged_data2, total_examples = data.shape[0], epochs = epoch)
        model.save(model_loc)
        print('New model training done. check--',model_loc)
    else:
        model=Doc2Vec.load(model_loc)
        embedding_df = pd.DataFrame(
            data = [model.infer_vector(doc.words) for doc in tagged_data2], 
            columns = ["d2v"+str(i) for i in range(d2v_size)])
        return embedding_df

In [6]:
# # Doc2Vec 임베딩 생성
d2v_data = df3[['BpID','tmp_x_items']]
file_dir = './Trained_models/Models_Doc2Vec/skts_model_doc2vec_%s.model' %(datetime.datetime.now().strftime("%y%m%d_%H%M"))
d2v_embedding = Doc2Vec(data=d2v_data, d2v_size=300, epoch=500, trainTF=False, model_loc=file_dir)

# 기존데이터에 Doc2Vec 처리된 데이터 추가
df4 = pd.merge(df3, d2v_embedding, left_index=True, right_index = True)

In [ ]:
# 크로스 컬럼
def CrossColumnGenerator(data, threshold = 5):
    """
    data : Data format must be pandas Series with rows having list of items.
              reset_index() required.
              example : '6601000010,6601000024,....,6601000001'
    threshold : To exclude super sparse case of cross columns
    dependencies : collections.Counter
                              sklearn.feature_extraction.text.CountVectorizer
    """
    cross_cnt = Counter()
    for ii, jj in data.iteritems():
        litem = sorted(set(jj.split(',')))
        cross_cnt.update([str(kk + '_' + ll) for kk, ll in combinations(litem, 2)])
        
    target_lcross = set({x : cross_cnt[x] for x in cross_cnt if cross_cnt[x] >= threshold}.keys()) 
    
    cross_feature = []
    for ii, jj in X_data.tmp_x_items.iteritems():
        litem = sorted(set(jj.split(',')))
        lcross = set([str(kk + '_' + ll) for kk, ll in combinations(litem, 2)]).intersection(target_lcross)
        cross_feature.append(lcross)

    lcross_feature = map(lambda x:' '.join(x), cross_feature)
    vect = CountVectorizer(tokenizer=str.split)
    mcross_feature = vect.fit_transform(lcross_feature)
    return pd.DataFrame(mcross_feature.todense(), columns=vect.get_feature_names())

# 크로스 컬럼 생성
# x_cross_embed = CrossColumnGenerator(X_data.tmp_x_items, threshold=1000)

In [ ]:
# 컬럼명 정리
x_else = ['ConstructionType', 'BusinessType', 'Regionnm']
x_d2v = ["d2v"+str(ii) for ii in range(300)]

x_item_colnames = x_item_embed_1.columns.tolist()
x_cross_colnames = x_cross_embed.columns.tolist()
y_item_colnames = y_item_embed.columns.tolist()

In [ ]:
# 데이터 뒤섞기
# 테이블 별로 따로 셔플햇기 때문에 인덱스가 꼬일수 있겠으나
# random_state = 0으로 햇기 때문에 모두 같은 index로 셔플되었다? 검증!
X_data = X_data.sample(frac = 1, random_state=0).reset_index(drop = True)
x_cross_embed = x_cross_embed.sample(frac = 1, random_state=0).reset_index(drop = True)
x_item_embed_1 = x_item_embed_1.sample(frac = 1, random_state=0).reset_index(drop = True)
y_item_embed = y_item_embed.sample(frac = 1, random_state=0).reset_index(drop = True)

In [ ]:
    if ii >= int(df_cnt*Train_volumn):
        tfrecords_name = "./Datasets/eval_dataset"+str(ii - int(df_cnt*Train_volumn))+".tfrecord"
    else:
        tfrecords_name = "./Datasets/train_dataset"+str(ii)+".tfrecord"


In [ ]:
# 분할 저장


In [ ]:
# 인풋 함수
X_else_keys = {
    "ConstructionType":X_data.ConstructionType.unique(), 
    "BusinessType":X_data.BusinessType.unique(), 
    "Regionnm":X_data.Regionnm.unique()
}

In [2]:
# def input_fn(data_files, num_epochs, batch_size):
#     def parse_csv(value):
#         columns = tf.decode_csv(value, record_defaults=iK['_csv_col_defaults'])
#         label = columns[-1]
#         del columns[-1]
#         features = columns
#         d = dict(zip(iK['featurenm'], features)), label
#         return d
#     # 최적화 이전 버전
#     dataset = (tf.data.TextLineDataset(data_files).skip(1).map(parse_csv, num_parallel_calls = 2))
#     dataset = dataset.prefetch(buffer_size = 50)
#     dataset = dataset.shuffle(400000)
#     dataset = dataset.repeat(num_epochs)
#     dataset = dataset.batch(batch_size)
#     iterator = dataset.make_one_shot_iterator()
#     batch_features, batch_labels = iterator.get_next()
#     return batch_features, batch_labels

# # files = tf.data.Dataset.list_files(data_files)
# # dataset = files.apply(tf.contrib.data.parallel_interleave(tf.data.TextLineDataset, cycle_length=4))
# # 또는 아래와 같은 방식? 위에는 베스트프랙티스 서머리에서 아래는 interleave 함수매뉴얼에서.
# # dataset = files.apply(tf.contrib.data.parallel_interleave(
# # lambda filename : tf.data.TextLineDataset(filename),cycle_length=4))
# #     dataset = dataset.shuffle(buffer_size)
# #     dataset = dataset.shuffle(buffer_size, reshuffle_each_iteration=True).repeat(num_epochs)
# #     dataset = dataset.prefetch(50) # datasize/batchsize

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/estimator/train_and_evaluate
# train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn)
# eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)

# tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

# test_dataset = full_dataset.skip(int(0.7 * DATASET_SIZE))
# val_dataset = test_dataset.skip(int(0.15 * DATASET_SIZE))
# test_dataset = test_dataset.take(int(0.15 * DATASET_SIZE))

tot_model.train(input_fn=train_input_fn)
# 181019 10:42시작 - 최종 메세지로 보이는 지속 시간.
# INFO:tensorflow:Calling model_fn 30분 -> 17분
# INFO:tensorflow:Done calling model_fn
# INFO:tensorflow:Create CheckpointSaverHook 10분
# INFO:tensorflow:Graph was finalized 5분
# INFO:tensorflow:Running local_init_op 
# INFO:tensorflow:Done running local_init_op 5분
# INFO:tensorflow:Saving checkpoints for 0 into ./Trained_models/Models_WnD/StartDate_181018/model.ckpt